### Split the data into training, test and validation set

In [1]:
cd("$(homedir())/Documents/enso_project.jl")
using Pkg
Pkg.activate(".")

  Activating project at `C:\Users\lihel\Documents\enso_project.jl`


In [6]:
using CSV, DataFrames

we use a function similiar to the one implemented in lecture 12.

In [16]:
"""
    train_val_test_split(data; num_val_months, num_test_months, Δt = 1)

Split the given data into training, validation, and test sets.
val_percent: number of time steps wanted in the validation set.
test_percent: number of time steps wanted in the test set.
"""
function train_val_test_split(data; val_percent=0.15, test_percent=0.15)
    N = size(data, 1)
    N_val = round(Int, val_percent*N)
    N_test = round(Int, test_percent*N)
    
    ind1 = N - N_test - N_val
    ind2 = N - N_test
    
    train_data = data[1:ind1, :]
    val_data = data[ind1+1:ind2, :]
    test_data = data[ind2+1:end, :]
    
    return train_data, val_data, test_data
end

train_val_test_split

we test it on our embedded data.

In [9]:
# read embedded data.
df_embed = CSV.read("data/sst_34_anomaly_embedded.txt", DataFrame; delim=',', ignorerepeated=true, header=0)

Row,Column1,Column2,Column3,Column4,Column5
,Float64,Float64,Float64,Float64,Float64
1,0.08,0.27,2.13,-0.33,-0.96
2,-0.2,0.86,1.81,-0.44,-0.16
3,-0.14,1.24,1.22,-0.38,-0.52
4,0.02,1.73,0.68,-1.02,-0.54
5,0.49,1.68,0.68,-1.29,-0.69
6,0.65,2.21,0.45,-1.16,-1.03
7,0.27,2.13,-0.33,-0.96,-0.68
8,0.86,1.81,-0.44,-0.16,-0.54
9,1.24,1.22,-0.38,-0.52,-0.41


In [18]:
train, val, test = train_val_test_split(df_embed)

(345×5 DataFrame
 Row │ Column1  Column2  Column3  Column4  Column5 
     │ Float64  Float64  Float64  Float64  Float64 
─────┼─────────────────────────────────────────────
   1 │    0.08     0.27     2.13    -0.33    -0.96
   2 │   -0.2      0.86     1.81    -0.44    -0.16
   3 │   -0.14     1.24     1.22    -0.38    -0.52
   4 │    0.02     1.73     0.68    -1.02    -0.54
   5 │    0.49     1.68     0.68    -1.29    -0.69
   6 │    0.65     2.21     0.45    -1.16    -1.03
   7 │    0.27     2.13    -0.33    -0.96    -0.68
   8 │    0.86     1.81    -0.44    -0.16    -0.54
  ⋮  │    ⋮        ⋮        ⋮        ⋮        ⋮
 339 │    1.11    -1.31    -0.83    -0.67    -0.44
 340 │    0.59    -1.41    -0.76    -0.71    -0.51
 341 │   -0.11    -1.44    -0.39    -0.98    -0.19
 342 │   -0.55    -1.21    -0.31    -0.99     0.24
 343 │   -0.92    -1.48    -0.19    -0.65     0.27
 344 │   -1.21    -1.19    -0.42    -0.55     0.68
 345 │   -1.31    -0.83    -0.67    -0.44     0.44
              

In [23]:
# store data
CSV.write("data/train_sst_34_anomaly_embedded.txt", train)
CSV.write("data/val_sst_34_anomaly_embedded.txt", val)
CSV.write("data/test_sst_34_anomaly_embedded.txt", test)

"data/test_sst_34_anomaly_embedded.txt"